<a href="https://colab.research.google.com/github/tanyajais04/copy_moveFrogeryDetection/blob/main/Test_cmfd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
import math
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
class DataGen(keras.utils.Sequence):
    def __init__(self, ids, path, batch_size=20, image_h=256, image_w = 256):
        self.ids = ids
        self.path = path
        self.batch_size = batch_size
        self.image_h = image_h
        self.image_w = image_w
        self.on_epoch_end()
        
    def __load__(self, id_name):
        ## Path
        image_path = os.path.join(self.path,'test_img', id_name)
        if '.png' in id_name:
            stringforid = id_name.strip('.png')
        if '.jpg' in id_name:
            stringforid = id_name.strip('.jpg')
        stringId = stringforid+'_mask.png'
        
        mask_path = os.path.join(self.path,'test_mask', stringId)      

        ## Reading Imag
        image = cv2.imread(image_path, 1)
        image = cv2.resize(image, (self.image_h, self.image_w))
        mask = np.zeros((self.image_h, self.image_w, 1))
           
        mask_img = cv2.imread(mask_path, 0)
        mask_img = cv2.resize(mask_img, (self.image_h, self.image_w))
        
        mask_img = np.expand_dims(mask_img, axis=-1)
        mask = np.maximum(mask, mask_img)
                
        ## Normalizaing 
        image = image/255.0
        mask = mask/255.0
        
        return image, mask
    
    def __getitem__(self, index):
        if(index+1)*self.batch_size > len(self.ids):
            self.batch_size = len(self.ids) - index*self.batch_size
        
        files_batch = self.ids[index*self.batch_size : (index+1)*self.batch_size]
        
        image = []
        mask  = []
        
        for id_name in files_batch:
            _img, _mask = self.__load__(id_name)
            image.append(_img)
            mask.append(_mask)
            
        image = np.array(image)
        mask  = np.array(mask)
        
        return image, mask
    
    def on_epoch_end(self):
        pass
    
    def __len__(self):
        return int(np.ceil(len(self.ids)/float(self.batch_size)))

In [ ]:
image_h = 256
image_w = 256

#change path of train with test and images also
#Path of test images
img_path = '/content/drive/MyDrive/CoMoFoD/Test'

epochs = 50

batch_size = 20

##Ids for test images 
ids = next(os.walk(os.path.join(img_path,'test_img')))[2]
data = DataGen(ids, img_path, image_h=image_h, image_w = image_w, batch_size=batch_size)


In [ ]:
def conv(x, filters, kernel_size=(3, 3), padding="same", strides=1):  
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides)(x)
    c = keras.layers.Activation('relu')(c)
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides)(c)
    c = keras.layers.Activation('relu')(c)
    return c

def max_pool(x):
    p = keras.layers.MaxPool2D((2, 2), (2, 2))(x)
    return p

def up_block(x, skip, filters, kernel_size=(3, 3), padding="same", strides=1):
    us = keras.layers.UpSampling2D((2, 2))(x)
    concat = keras.layers.Concatenate()([us, skip])
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides)(concat)
    c = keras.layers.Activation('relu')(c)
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides)(c)
    c = keras.layers.Activation('relu')(c)
    return c

def bottleneck(x, filters, kernel_size=(5, 5), padding="same", strides=1):
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides)(x)
    c = keras.layers.Activation('relu')(c)
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides)(c)
    c = keras.layers.Activation('relu')(c)
    return c

In [ ]:
def UNet():
    f = [16, 32, 64, 128, 256]
    image_size= 256
    s0 = keras.layers.Input((image_size, image_size, 3))
    pt0 = keras.layers.experimental.preprocessing.RandomRotation(0.2)(s0)
    
    s1 = keras.layers.MaxPool2D((2, 2), (2, 2))(s0)
    pt1 = keras.layers.experimental.preprocessing.RandomRotation(0.2)(s1)

    s2 = keras.layers.MaxPool2D((2, 2), (2, 2))(s1)
    pt2 = keras.layers.experimental.preprocessing.RandomRotation(0.2)(s2)

    s3 = keras.layers.MaxPool2D((2, 2), (2, 2))(s2)
    pt3 = keras.layers.experimental.preprocessing.RandomRotation(0.2)(s3)

    c0 = conv(pt0, f[0])
    p0 = max_pool(c0)
    c1 = conv(pt1, f[1])
    l1 = keras.layers.Concatenate()([p0,c1])
    p1 = max_pool(l1)
    
    c2 = conv(pt2, f[2])
    l2 = keras.layers.Concatenate()([p1,c2])
    p2 = max_pool(l2)

    c3 = conv(pt3, f[3])
    l3 = keras.layers.Concatenate()([p2,c3])
    p3 = max_pool(l3)
    
    bn = bottleneck(p3, f[4])
    
    u1 = up_block(bn, c3, f[3]) #8 -> 16
    u2 = up_block(u1, c2, f[2]) #16 -> 32
    u3 = up_block(u2, c1, f[1]) #32 -> 64
    u4 = up_block(u3, c0, f[0]) #64 -> 128
    
    outputs = keras.layers.Conv2D(1, (1, 1), padding="same", activation="sigmoid")(u4)
    model = keras.models.Model(s0, outputs)
    return model

In [ ]:
model = UNet()
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["acc"])

In [ ]:
#Load weight
model.load_weights('/content/drive/MyDrive/project dl/modelweight.h5')

In [ ]:
def evaluation_matrix(gt,r):
    tp = 0 
    fp = 0
    tn = 0
    fn = 0
    size = gt.shape
    m = size[0]
    n = size[1]
    for i in range(0,gt.shape[0]):
        for j in range(0,gt.shape[1]):
            if r[i][j]==0 and gt[i][j]==0:
                tp=tp+1
            elif r[i][j]==0 and gt[i][j]==1:
                fp = fp+1
            elif r[i][j]==1 and gt[i][j]==0:
                fn = fn+1
            elif r[i][j]==1 and gt[i][j]==1:
                tn = tn+1
    #prec = tp/(tp+fp)
    #recall = tp/(tp+fn)
    #acc = (tp+tn)/(tp+tn+fp+fn)
    #tnr = tn/(tn+fp)
    #fpr = fp/(fp+tn)
    #fnr = fn/(fn+tp)
    #csi = tp/(tp+fn+fp)
    #f1 = 2*((prec*recall)/(prec+recall))
    #mcc = (tp*tn-fp*fn)/math.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))
    return tp,tn,fp,fn
#prec,recall,acc,tnr,fnr,f1,mcc

In [ ]:
x, y = data.__getitem__(6)
result1 = model.predict(x)

In [ ]:
result = result1>0.5

In [ ]:
  for j in range(0,20):
    fig = plt.figure(figsize=(20,10))

    fig.subplots_adjust(hspace=0.4, wspace=0.4)

    ax = fig.add_subplot(1, 3, 1)
    b,g,r = cv2.split(x[j])       # get b,g,r
    rgb_img = cv2.merge([r,g,b]) 
    ax.imshow(rgb_img)

    ax = fig.add_subplot(1, 3, 2)
    ax.imshow(np.reshape(y[j]*255, (image_h, image_w)), cmap="gray")

    ax = fig.add_subplot(1, 3, 3)
    ax.imshow(np.reshape(result[j]*255, (image_h, image_w)), cmap="gray")
    
    tp,tn,fp,fn = evaluation_matrix(y[j],result[j])
    print(str(tp)+','+str(tn)+','+str(fp)+','+str(fn))